In [1]:
import os

# 更改缓存路径
os.environ["HF_HOME"] = "D:/huggingface"
os.environ["HF_DATASETS_CACHE"] = "D:/huggingface/datasets"

![](https://chushi123.oss-cn-beijing.aliyuncs.com/img/202203051058629.png)

在将文本拆分为子标记之前（根据其模型），标记器执行两个步骤：规范化和预标记化。

规范化normalization步骤涉及一些常规清理，例如删除不必要的空格、统一为小写和删除重音符号。如果您熟悉Unicode 规范化（例如 NFC 或 NFKC），这也是标记器可能应用的东西。

不同的模型架构，规范化过程是不一样的。

![](https://chushi123.oss-cn-beijing.aliyuncs.com/img/202203051137435.png)

# normalization

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
print(type(tokenizer.backend_tokenizer))

<class 'tokenizers.Tokenizer'>


🤗 Transformerstokenizer有一个名为backend_tokenizer的属性，它提供对来自 🤗 Tokenizers 库的底层标记器的访问：

tokenizers.Tokenizer对象的normalizer属性有一个normalize_str()方法，我们可以使用它来查看normalization是如何执行的：

在此示例中，由于我们选择了bert-base-uncased检查点，因此normalization应用了小写字母并删除了重音符号。

In [3]:
print(tokenizer.backend_tokenizer.normalizer.normalize_str("Héllò hôw are ü?"))

hello how are u?


#  pre-tokenization

pre-tokenization应用一些规则实现了对文本的初步切分。

![](https://chushi123.oss-cn-beijing.aliyuncs.com/img/202203051154902.png)

不同模型的pre-tokenization是不一样的。

![](https://chushi123.oss-cn-beijing.aliyuncs.com/img/202203051157963.png)

In [8]:
# 同一个output里显示多个输出结果
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

## bert的分词器

In [5]:
tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str("Hello, how are  you?")
tokenizer("Hello, how are  you?").tokens()

[('Hello', (0, 5)),
 (',', (5, 6)),
 ('how', (7, 10)),
 ('are', (11, 14)),
 ('you', (16, 19)),
 ('?', (19, 20))]

['[CLS]', 'hello', ',', 'how', 'are', 'you', '?', '[SEP]']

bert的分词器会将多个空格处理为一个空格，are和you的索引跳跃可以说明这一点。

## GPT-2的分词器

In [6]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str("Hello, how are  you?")
tokenizer("Hello, how are  you?").tokens()

[('Hello', (0, 5)),
 (',', (5, 6)),
 ('Ġhow', (6, 10)),
 ('Ġare', (10, 14)),
 ('Ġ', (14, 15)),
 ('Ġyou', (15, 19)),
 ('?', (19, 20))]

['Hello', ',', 'Ġhow', 'Ġare', 'Ġ', 'Ġyou', '?']

GPT-2也会在空格和标点符号上拆分，但它会保留空格并用Ġ符号替换它们。

## T5的分词器基于SentencePiece的Unigram subword tokenization algorithms

In [7]:
tokenizer = AutoTokenizer.from_pretrained("t5-small")
tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str("Hello, how are  you?")
tokenizer("Hello, how are  you?").tokens()

[('▁Hello,', (0, 6)),
 ('▁how', (7, 10)),
 ('▁are', (11, 14)),
 ('▁you?', (16, 20))]

['▁Hello', ',', '▁how', '▁are', '▁you', '?', '</s>']

T5分词器与 GPT-2 分词器一样，此分词器保留空格并用特定分号 （_） 替换它们。另请注意，默认情况下，它在句子的开头（在 Hello 之前）添加了一个空格，并忽略了 are 和 you 之间的双倍空格。